In [1]:
import gc
import sys
import numpy as np
import numpy.random as rand
import math
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.metrics import roc_curve, auc, roc_auc_score
from scipy import stats
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers
# import keras as keras
# from keras.models import Sequential, load_model
# from keras.layers import Dense, Activation, Dropout
# from keras.callbacks import EarlyStopping, ReduceLROnPlateau
# from keras import regularizers
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split
#import pickle as pickle

# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# sess = tf.Session(config=config)
import time
import glob
import numpy.ma as ma
from tensorflow.keras import backend as K
# K.set_session(sess)

from cwola_utils import AddPredictionsToScatter_nestedcrossval
from cwola_utils import model_ensemble
from cwola_utils import check_eff
from cwola_utils import print_scatter_checkpoint
from cwola_utils import get_p_value

rand.seed(0)
#sys.stdout = os.fdopen(sys.stdout.fileno(), 'w', 0)

from tensorflow.python.client import device_lib
print("Devices: ", device_lib.list_local_devices())


1 Physical GPUs, 1 Logical GPUs
Devices:  [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3220151433632241012
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4236279702372032591
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 9883535296
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4318807640692725893
physical_device_desc: "device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13369019576538894687
physical_device_desc: "device: XLA_GPU device"
]


In [2]:
import numpy as np
np.round(np.logspace(1, 6, num=18))

array([1.00000e+01, 2.00000e+01, 3.90000e+01, 7.60000e+01, 1.50000e+02,
       2.96000e+02, 5.82000e+02, 1.14500e+03, 2.25400e+03, 4.43700e+03,
       8.73300e+03, 1.71910e+04, 3.38390e+04, 6.66080e+04, 1.31113e+05,
       2.58086e+05, 5.08022e+05, 1.00000e+06])

In [18]:

def getopts(argv):
    opts = {}  # Empty dictionary to store key-value pairs.
    while len(argv)>1:  # While there are arguments left to parse...
        if argv[0][0] == '-':  # Found a "-name value" pair.
            opts[argv[0]] = argv[1]  # Add key and value to the dictionary.
        argv = argv[1:]  # Reduce the argument list by copying it starting from index 1.
    return opts

myargs = getopts(sys.argv)
print("Command line arguments: ", myargs)
if '-o' not in myargs:
    print("Setting default output name '-o': '~/model'")
    myargs['-o'] = "~/model"
if '-in' not in myargs:
    print("Setting default input directory to '/data1/users/jcollins/'")
    myargs['-in'] = '/data1/users/jcollins/'
if '-bin' not in myargs:
    myargs['-bin'] = 7
    print("Setting default bin '-bin':", myargs['-bin'])
else:
    myargs['-bin'] = int(myargs['-bin'])
if '-it' not in myargs:
    myargs['-it'] = 3
    print("Setting default  '-it':", myargs['-it'])
else:
    myargs['-it'] = int(myargs['-it'])
    
if '-kfold' not in myargs:
    myargs['-kfold'] = 5
    print("Setting default  '-kfold':",myargs['-kfold'])
else:
    myargs['-kfold'] = int(myargs['-kfold'])
    
if '-signal' not in myargs:
    myargs['-signal'] = 1
    print("Setting default  '-signal':",myargs['-signal'])
else:
    myargs['-signal'] = int(myargs['-signal'])
    
if '-sigevnts' not in myargs:
    myargs['-sigevnts'] = 1125
    print("Setting default  '-sigevnts':",myargs['-sigevnts'])
else:
    myargs['-sigevnts'] = int(myargs['-sigevnts'])
if '-bgevnts' not in myargs:
    myargs['-bgevnts'] = 553388
    print("Setting default  '-bgevnts':", myargs['-bgevnts'])
else:
    myargs['-bgevnts'] = int(myargs['-bgevnts'])
if '-bgoffset' not in myargs:
    myargs['-bgoffset'] = 0
    print("Setting default  '-bgoffset':", myargs['-bgoffset'])
else:
    myargs['-bgoffset'] = int(myargs['-bgoffset'])

if '-batchsize' not in myargs:
    myargs['-batchsize'] = 5000
    print("Setting default  '-batchsize':", myargs['-batchsize'])
else:
    myargs['-batchsize'] = int(myargs['-batchsize'])
if '-patience' not in myargs:
    myargs['-patience'] = 250
    print("Setting default  '-patience':", myargs['-patience'])
else:
    myargs['-patience'] = int(myargs['-patience'])
if '-checkeff' not in myargs:
    myargs['-checkeff'] = 0.01
    print("Setting default  '-checkeff':", myargs['-checkeff'])
else:
    myargs['-checkeff'] = float(myargs['-checkeff'])

if '-loadonly' not in myargs:
    myargs['-loadonly'] = 0
    print("Setting default  '-loadonly':", myargs['-loadonly'])
else:
    myargs['-loadonly'] = float(myargs['-loadonly'])
if '-trainonly' not in myargs:
    myargs['-trainonly'] = 0
    print("Setting default  '-trainonly':", myargs['-trainonly'])
else:
    myargs['-trainonly'] = int(myargs['-trainonly'])

    
if '-startk' not in myargs:
    myargs['-startk'] = 0
    print("Setting default  '-startk':", myargs['-startk'])
else:
    myargs['-startk'] = int(myargs['-startk'])
if '-startl' not in myargs:
    myargs['-startl'] = 0
    print("Setting default  '-startl':", myargs['-startl'])
else:
    myargs['-startl'] = int(myargs['-startl'])
if '-endk' not in myargs:
    myargs['-endk'] = 0
    print("Setting default  '-endk':", myargs['-endk'])
else:
    myargs['-endk'] = int(myargs['-endk'])
if '-endl' not in myargs:
    myargs['-endl'] = 0
    print("Setting default  '-endl':", myargs['-endl'])
else:
    myargs['-endl'] = int(myargs['-endl'])
    
myargs['-in'] = 'E:/projects/CWoLa-Hunting/data/'
myargs['-o'] = 'E:/projects/CWoLa-Hunting/trained_models'

bin_i = myargs['-bin']    
data_prefix = myargs['-in']
output_prefix = myargs['-o']


Command line arguments:  {'--ip=127.0.0.1': '--stdin=9053', '--stdin=9053': '--control=9051', '--control=9051': '--hb=9050', '--hb=9050': '--Session.signature_scheme="hmac-sha256"', '--Session.signature_scheme="hmac-sha256"': '--Session.key=b"2979bbf3-a28c-41e5-8bca-b5dc82f00227"', '--Session.key=b"2979bbf3-a28c-41e5-8bca-b5dc82f00227"': '--shell=9052', '--shell=9052': '--transport="tcp"', '--transport="tcp"': '--iopub=9054', '--iopub=9054': '--f=C:\\Users\\Jack\\AppData\\Local\\Temp\\tmp-14268GgiUEIqU7uGQ.json'}
Setting default output name '-o': '~/model'
Setting default input directory to '/data1/users/jcollins/'
Setting default bin '-bin': 7
Setting default  '-it': 3
Setting default  '-kfold': 5
Setting default  '-signal': 1
Setting default  '-sigevnts': 1125
Setting default  '-bgevnts': 553388
Setting default  '-bgoffset': 0
Setting default  '-batchsize': 5000
Setting default  '-patience': 250
Setting default  '-checkeff': 0.01
Setting default  '-loadonly': 0
Setting default  '-tra

In [19]:
#Which auxilliary variables to use
selected_vars = [1,4,2,5,3,6]
#Also use mJJ.
selected_vars_plus = np.append([0],selected_vars)

#Which 2d planes to make scatter plots in
axes_list = [[1,2],[3,4],[5,6]]
axes_labels = [['mJA','mJB'],['tau_21A','tau21B'],['tau_32A','tau32B']]

#Data binning in mJJ
mjjmin = 2500
mjjmax = 6000
mybinboundaries = np.round(np.logspace(np.log10(mjjmin), np.log10(mjjmax), num=18))
mybincenters = np.array([0.5*(mybinboundaries[i+1] + mybinboundaries[i]) for i in range(0,len(mybinboundaries)-1)])
mybinwidths = np.array([mybinboundaries[i+1] - mybinboundaries[i] for i in range(0,len(mybinboundaries)-1)])
mybincenterandwidth = np.vstack((mybincenters,mybinwidths)).T

def bin_data(data, binboundaries = mybinboundaries):
    databinned = []
    for i in range(0,len(binboundaries)-1):
        databinned.append(
            np.array([myrow for myrow in data if (myrow[0] < binboundaries[i+1] and myrow[0] >= binboundaries[i])])
        )
    return databinned

import pandas as pd
#load data
print('\n')
print("Loading E:\projects\CWoLa-Hunting\LHCO_data\events_anomalydetection_v3.features.h5")
f = pd.read_hdf("E:/projects/CWoLa-Hunting/LHCO_data/events_anomalydetection_v3.features.h5")
data = f.values
E1 = np.sqrt(np.square(np.linalg.norm(data[:,:3],axis=-1))-np.square(data[:,3]))
E2 = np.sqrt(np.square(np.linalg.norm(data[:,7:10],axis=-1))-np.square(data[:,10]))
mjj = np.sqrt(np.square(E1+E2)-np.square(np.linalg.norm(data[:,:3]+data[:,7:10],axis=-1)))
bg_plus_signal = np.concatenate(
    (mjj[:,None],data[:,3:4],data[:,5:6]/data[:,4:5],data[:,6:7]/data[:,5:6],
    data[:,10:11],data[:,12:13]/data[:,11:12],data[:,13:14]/data[:,12:13]), axis=-1
)
bg_plus_signal = np.nan_to_num(bg_plus_signal)
bg_plus_signal = bg_plus_signal[np.greater(mjj,mjjmin) & np.less(mjj,mjjmax)]

# Preprocessor for data before feeding it to NN
#   1) Take log off jet mass variables
#   2) Standard scale all auxilliary variables
def myprepreprocessor(predata, massvars=None):
    if massvars is None:
        massvars = [0,1]
    newdata = np.copy(predata)
    newdata[:,massvars] = np.nan_to_num(np.log10((newdata[:,massvars]-30.)/newdata[:,0:1]+30./3000.))
    return newdata[:,1:]


bg_plus_signal=bg_plus_signal[:,selected_vars_plus]
myscaler = preprocessing.StandardScaler().fit(myprepreprocessor(bg_plus_signal,massvars=[1,2]))
def preprocess(data):
    return np.clip(myscaler.transform(myprepreprocessor(data,massvars=[0,1])),-3,3)
rand.shuffle(bg_plus_signal)

bg_plus_signal_binned = bin_data(bg_plus_signal)
del bg_plus_signal

import warnings
warnings.filterwarnings('ignore')

ntries = myargs['-it']
times = list()
print("Working on bin ", bin_i)

# This is leftover from earlier days when we experimented with removing jet mass from the auxilliary variables to make them dimensionless.
no_mass = False
numvars=len(selected_vars)
if no_mass:
    numvars = numvars-2


    
# I have found that large batch sizes ~5000 work well for our problem. I have not done a systematic test or scan. But my intuition is that because signal events are so rare, even in the signal region, you want batch sizes large enough so that most batches will contain at least a few true signal events.
batch_size = myargs['-batchsize']

model_utils = {}
model_utils[bin_i] = model_ensemble(bg_plus_signal_binned, bin_i = bin_i, kfolds=myargs['-kfold'], preprocess = preprocess, eff_for_thresh = myargs['-checkeff'])

gc.collect()




Loading E:\projects\CWoLa-Hunting\LHCO_data\events_anomalydetection_v3.features.h5
Working on bin  7


14352

In [37]:
k=0
model_utils = {}

for bin_i in range(3,11):


    model_utils[bin_i] = model_ensemble(bg_plus_signal_binned, bin_i = bin_i, kfolds=myargs['-kfold'], preprocess = preprocess, eff_for_thresh = myargs['-checkeff'])

    myargs['-loadonly'] = True
    #Loop over validation sets
    for l in range(myargs['-kfold']):
        if (myargs['-endl'] > 0) & (l == myargs['-endl']):
            print("Ending at l = ", l)
            sys.stdout.flush()
            quit()
        if l == k:
            continue
        print('Starting lfold', l, 'of', myargs['-kfold'])
        
        data_train, data_valid, labels_train, labels_valid, weights_train, weights_valid = model_utils[bin_i].get_trainval_data(k,l)

        for i in range(ntries):
            print(" k =", k, "l =", l)
            #Naming convention for model files.
            checkpoint_name = output_prefix + "_" + str(bin_i) + "_[" + str(k) + "," + str(l) + "]_" + str(i)
            start = time.time()
            
            #The flag myargs['-loadonly'] allows to skip training step and just load pre-saved models.
            if (not myargs['-loadonly']) & (not ((k == myargs['-startk']) & (l < myargs['-startl']))) & (k >= myargs['-startk']):
                print("Now training model ", i + 1, " of ", ntries)
                
                K.clear_session()
                #Following hyperparams seem to work well. Not done systematic optimization. Maybe something else works much better.
                myoptimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.8, beta_2=0.99, epsilon=1e-08, decay=0.0005)
                
                #Custom callback to record tpr at fixed fpr (set by eff_rate), where tpr and fpr refer to signal and sideband regions rather than truth-labels.
                my_check_eff = check_eff(verbose=0, filename = checkpoint_name + '_best.h5', patience = myargs['-patience'],
                                            preprocessed_training_data=(preprocess(data_train),labels_train),
                                            min_epoch=10,
                                            plot_period=2,eff_rate=myargs['-checkeff'],
                                            validation_data = (preprocess(data_valid), labels_valid, weights_valid))
                #Custom callback for printing scatter plots every few epochs. Useful for troubleshooting, but slows down training considerably.
                my_print = print_scatter_checkpoint(filename = checkpoint_name,
                                                    axes_list = axes_list,
                                                    axes_labels = axes_labels,
                                                    period=20,
                                                    training_data=np.append(data_train,data_valid,axis=0),
                                                    preprocess=preprocess)
                
                mycallbacks=[#my_print,
                    my_check_eff]
                
                #Following seems to work well for benchmarks. Not systematically optimized. I basically just played around until something worked.
                #However, bias initialization seems very important. Keras relu by default initializes to 0 bias, and especially in the first layer will not move from that initialization during training. This is very suboptimal.
                model = Sequential(name = 'checkpoint_name')
                model.add(Dense(128, input_dim=numvars,use_bias=True,
                                #activation='relu',
                                bias_initializer = keras.initializers.TruncatedNormal(mean=0., stddev=0.04)))
                model.add(keras.layers.LeakyReLU(alpha=0.1))
                model.add(Dropout(0.1))
                model.add(Dense(128, use_bias=True, activation='elu',
                                bias_initializer = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01)))
                model.add(Dropout(0.1))
                model.add(Dense(128, use_bias=True, activation='elu',
                                bias_initializer = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01)))
                model.add(Dropout(0.1))
                model.add(Dense(128, use_bias=True, activation='elu',
                                bias_initializer = keras.initializers.TruncatedNormal(mean=0.0, stddev=0.01)))
                model.add(Dense(1, activation='sigmoid'))
                
                model.compile(optimizer=myoptimizer,
                                loss='binary_crossentropy')
                
                model_hist = model.fit(preprocess(data_train), labels_train, epochs=2000, batch_size=batch_size,
                                        validation_data=(preprocess(data_valid), labels_valid, weights_valid),
                                        callbacks=mycallbacks,verbose=0,
                                        sample_weight = weights_train)
                
                del model
                K.clear_session()           #Otherwise TensorFlow eats up all GPU memory with previous models.
            else:
                print("Now loading model ", i + 1, " of ", ntries)
                
            model = keras.models.load_model(checkpoint_name + "_best.h5")
            model._name = str(bin_i) + "_" + str(k) + "_" + str(l) + "_" + str(i)
            model.save(checkpoint_name + "_best.h5")
            model_utils[bin_i].add_model(model, None, k, l,checkpoint_name + "_best.h5")
            plt.close('all')
        
            if (not myargs['-loadonly']) & (not ((k == myargs['-startk']) & (l < myargs['-startl']))) & (k >= myargs['-startk']):
                model_utils[bin_i].print_scatter_onemodel_signalregion(k,l,i,axes_list=axes_list,axes_labels=axes_labels,
                                                                        rates = np.array([1. - myargs['-checkeff']]),
                                                                        colors=['silver','firebrick'])
                figfile=checkpoint_name + '_scatterk_' + str(k) + '.png'
                print('Saving fig', figfile)
                plt.savefig(figfile)
                
            for i in range(5):
                gc.collect()
            end = time.time()
            
            times.append(end-start)
            print("Elapsed Time = ", times[-1])
            sys.stdout.flush()
            
    print("Bin = ", bin_i)
    print("aucs valid: ", model_utils[bin_i].aucs_valid)
    print("Effs valid: ", model_utils[bin_i].effs_valid)
    print("aucs train: ", model_utils[bin_i].aucs_train)
    print("Effs train: ", model_utils[bin_i].effs_train)
    print("\n")

    #Make an ensemble model using the average of the best models trained using the (k-1) training-validation splits. Save this as a single model.
    ensemble_model = model_utils[bin_i].makeandsave_ensemble_model(k,output_prefix + "_" + str(bin_i) + "_ensemble_k" + str(k) + ".h5")
    plt.close('all')
    model_utils[bin_i].print_scatter_avg_onek_signalregion(k,axes_list=axes_list,axes_labels=axes_labels,
                                                            rates = np.array([1. - myargs['-checkeff']]),
                                                            colors=['silver','firebrick'])
    figfile=output_prefix + '_scatterk_' + str(bin_i) + '_' + str(k) + '.png'
    print('Saving fig', figfile)
    plt.savefig(figfile)
    del ensemble_model
    K.clear_session()
    sys.stdout.flush()



Starting lfold 1 of 5
 k = 0 l = 1
Now loading model  1  of  3
Elapsed Time =  1.6318323612213135
 k = 0 l = 1
Now loading model  2  of  3
Elapsed Time =  1.5362093448638916
 k = 0 l = 1
Now loading model  3  of  3
Elapsed Time =  1.5112102031707764
Starting lfold 2 of 5
 k = 0 l = 2
Now loading model  1  of  3
Elapsed Time =  1.5622766017913818
 k = 0 l = 2
Now loading model  2  of  3
Elapsed Time =  1.5367212295532227
 k = 0 l = 2
Now loading model  3  of  3
Elapsed Time =  1.5282342433929443
Starting lfold 3 of 5
 k = 0 l = 3
Now loading model  1  of  3
Elapsed Time =  1.5173048973083496
 k = 0 l = 3
Now loading model  2  of  3
Elapsed Time =  1.534104347229004
 k = 0 l = 3
Now loading model  3  of  3
Elapsed Time =  1.5902023315429688
Starting lfold 4 of 5
 k = 0 l = 4
Now loading model  1  of  3
Elapsed Time =  1.5766911506652832
 k = 0 l = 4
Now loading model  2  of  3
Elapsed Time =  1.561037302017212
 k = 0 l = 4
Now loading model  3  of  3
Elapsed Time =  1.562300443649292
Bin

In [34]:
model.name

'7_0_4_2'

In [26]:
model._name = 'bogfdgdb'

In [27]:
model.name

'bogfdgdb'